<a href="https://colab.research.google.com/github/MUmairAB/BERT-based-NER-using-HuggingFace-Transformers/blob/main/BERT_NER_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install the transformers library
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00


In [2]:
#Install the datasets library
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 5.5 MB/s eta 0:00:00


## Dataset

For training the model, we'll use [this](https://huggingface.co/datasets/conll2003) dataset from the HuggingFace Datasets.

In [3]:
#Load the dataset
from datasets import load_dataset
data = load_dataset(path="conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
#Inspect the Dataset
data

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
#Interview the data
data["train"][100]["tokens"]

['Rabinovich', 'is', 'winding', 'up', 'his', 'term', 'as', 'ambassador', '.']

In [6]:
#The corresponding "ner_tag" data
data["train"][100]["ner_tags"]

[1, 0, 0, 0, 0, 0, 0, 0, 0]

In [7]:
#The corresponding labels for the integer tags
ner_labels = data["train"].features["ner_tags"]
ner_labels

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [8]:
#Convert the Sequence object to a simple list
ner_labels_list = ner_labels.feature.names
ner_labels_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [9]:
#Print the full sentence and their corresponding NER-tags
words = data["train"][0]["tokens"]
labels = data["train"][0]["ner_tags"]
sentence = ""
ner_tagged_sen = ""

for word, label in zip(words, labels):
    #Get the corresponding label for an integer label
    full_label = ner_labels_list[label]
    #Find the maximum length from these two
    max_length = max(len(word), len(full_label))
    sentence += word + " " * (max_length - len(word) + 1)
    ner_tagged_sen += full_label + " " * (max_length - len(full_label) + 1)

print(sentence)
print(ner_tagged_sen)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


## Vectorization

In [10]:
#Instantiate the tokenizer
from transformers import AutoTokenizer
checkpoints = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoints)

In [11]:
#Define a function to align the labels with the tokens
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [12]:
#Define a function to tokenize the raw text and apply the
# "align_labels_with_tokens" function
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [13]:
#Use the Dataset.map() method to apply tokenize_and_align_labels
# function on all the dataset
tokenized_datasets = data.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=data["train"].column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [14]:
#Apply the data collation
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, return_tensors="tf"
)

In [15]:
#Use the datacollator to create tf.data.Dataset Object
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)

tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


## Model

In [16]:
#Define the mapping from ID to label and vice versa
id2label = {i: label for i, label in enumerate(ner_labels_list)}
label2id = {v: k for k, v in id2label.items()}

In [17]:
#Instantiate the model
from transformers import TFAutoModelForTokenClassification
model = TFAutoModelForTokenClassification.from_pretrained(
    pretrained_model_name_or_path=checkpoints,
    id2label=id2label,
    label2id=label2id
                    )

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
#Cross verify the number of labels
model.config.num_labels

9

In [19]:
#Some properties of the model architecture

print("Some Properties of the Model Architecture")
print("\tNumber of Hidden Layers = ",model.config.num_hidden_layers)

print("\tNumber of Attention Heads = ",model.config.num_attention_heads)

print("\tThe model type = ",model.config.model_type)

print("\tVocabulary size = ",model.config.vocab_size)

Some Properties of the Model Architecture
	Number of Hidden Layers =  12
	Number of Attention Heads =  12
	The model type =  bert
	Vocabulary size =  28996


In [20]:
#Print the model summary
model.summary()

Model: "tf_bert_for_token_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  107719680 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  6921      
                                                                 
Total params: 107,726,601
Trainable params: 107,726,601
Non-trainable params: 0
_________________________________________________________________


In [21]:
#@title
#Log in to the HF account
from huggingface_hub import notebook_login

notebook_login()

In [22]:
#Define the optimizer
from transformers import create_optimizer
import tensorflow as tf

num_epochs = 20
#Calculate the number of training steps using the formula
#(num_samples // batch_size) * num_epochs
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

#Compile the model
model.compile(optimizer=optimizer)

In [23]:
#Define the callbacks
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(output_dir="bert-ner", tokenizer=tokenizer)

Cloning https://huggingface.co/MUmairAB/bert-ner into local empty directory.


In [24]:
#Train the model
model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=[callback],
    epochs=num_epochs,
)

Epoch 1/20
878/878 [==============================] - 226s 232ms/step - loss: 0.1775 - val_loss: 0.0635
Epoch 2/20
878/878 [==============================] - 230s 262ms/step - loss: 0.0470 - val_loss: 0.0559
Epoch 3/20
878/878 [==============================] - 231s 263ms/step - loss: 0.0278 - val_loss: 0.0603
Epoch 4/20
878/878 [==============================] - 227s 258ms/step - loss: 0.0174 - val_loss: 0.0603
Epoch 5/20
878/878 [==============================] - 226s 258ms/step - loss: 0.0124 - val_loss: 0.0615
Epoch 6/20
878/878 [==============================] - 221s 252ms/step - loss: 0.0077 - val_loss: 0.0722
Epoch 7/20
878/878 [==============================] - 225s 256ms/step - loss: 0.0060 - val_loss: 0.0731
Epoch 8/20
878/878 [==============================] - 235s 268ms/step - loss: 0.0038 - val_loss: 0.0757
Epoch 9/20
878/878 [==============================] - 231s 263ms/step - loss: 0.0043 - val_loss: 0.0731
Epoch 10/20
878/878 [==============================] - 236s 269m

## Model Evaluation

In [25]:
#Install the seqeval library
!pip install seqeval
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=b3e14ae9d62cec6e28d33a119a653537726c28532a5a95dabd4c62d0dc69226f
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.7 MB/s eta 0:00:00


In [26]:
#Import the seqeval using evaluate
import evaluate

metric = evaluate.load("seqeval")

In [27]:
#Evaluate the model predictions on fake data
import numpy as np

all_predictions = []
all_labels = []
for batch in tf_eval_dataset:
    logits = model.predict_on_batch(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    for prediction, label in zip(predictions, labels):
        for predicted_idx, label_idx in zip(prediction, label):
            if label_idx == -100:
                continue
            all_predictions.append(ner_labels_list[predicted_idx])
            all_labels.append(ner_labels_list[label_idx])
metric.compute(predictions=[all_predictions], references=[all_labels])

{'LOC': {'precision': 0.9655361050328227,
  'recall': 0.9608056614044638,
  'f1': 0.9631650750341064,
  'number': 1837},
 'MISC': {'precision': 0.8789144050104384,
  'recall': 0.913232104121475,
  'f1': 0.8957446808510638,
  'number': 922},
 'ORG': {'precision': 0.9075144508670521,
  'recall': 0.9366144668158091,
  'f1': 0.9218348623853211,
  'number': 1341},
 'PER': {'precision': 0.962011771000535,
  'recall': 0.9761129207383279,
  'f1': 0.9690110482349771,
  'number': 1842},
 'overall_precision': 0.9374068554396423,
 'overall_recall': 0.9527095254123191,
 'overall_f1': 0.944996244053084,
 'overall_accuracy': 0.9864013657502796}